In [2]:
import vtk
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from vtk.util import numpy_support

In [25]:
def vti_reader(file_path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    data = reader.GetOutput()
    
    # vtk res
    dataset = pd.DataFrame(np.array(
    [numpy_support.vtk_to_numpy(data.GetPointData().GetArray(i))
    for i in range(data.GetPointData().GetNumberOfArrays())])).transpose()
    
    dataset.columns=[data.GetPointData().GetArrayName(i) for i in range(data.GetPointData().GetNumberOfArrays())]
    
    # vtk coord
    dimensions = data.GetDimensions()
    spacing=data.GetSpacing()
    origin=data.GetOrigin()
    
    coordinates=[]
    
    for k in range(dimensions[2]):
        for j in range(dimensions[1]):
            for i in range(dimensions[0]):
                x = origin[0] + i * spacing[0]
                y = origin[1] + j * spacing[1]
                z = origin[2] + k * spacing[2]
                coordinates.append((x,y,z))
    dt_coordinates = pd.DataFrame(coordinates, columns=['x','y','z'])
    
    # result coord merge
    return pd.merge(dt_coordinates, dataset, left_index=True, right_index=True)

In [26]:
file_path = './outputs/twophase_coat_v5_19/window_0000/inferencers/time_slice_000t.vti'

In [29]:
vti_data = vti_reader(file_path)

In [30]:
vti_data.head()

,x,y,z,u,v,p,a,alpha,mu
0,-25.000000,0.0,0.0,0.062075,0.009651,-0.001931,1.011247,1.0,-0.056816
1,-24.389764,0.0,0.0,0.061572,0.008842,-0.001132,1.011040,1.0,-0.055269
2,-23.779528,0.0,0.0,0.061073,0.008088,-0.000022,1.010967,1.0,-0.054440
3,-23.169291,0.0,0.0,0.060366,0.007212,0.001040,1.010779,1.0,-0.053083
4,-22.559055,0.0,0.0,0.059611,0.006587,0.001857,1.010449,1.0,-0.051032


In [47]:
vti_slurry = vti_data[vti_data['alpha']==0.0]
pnt1 = vti_slurry['x'].min()

In [51]:
pnt2 = vti_slurry[vti_slurry['y']==0.0]['x'].min()

In [52]:
pnt3 = vti_slurry[vti_slurry['y'] == vti_slurry['y'].max()]['x']

In [54]:
pnt1, pnt2, pnt3

(-1.2007874015748037,
 -0.5905511811023629,
 12969    0.019685
 12970    0.629921
 12971    1.240157
 12972    1.850394
 12973    2.460630
 Name: x, dtype: float64)

In [55]:
 vti_slurry[vti_slurry['y'] == vti_slurry['y'].max()]

,x,y,z,u,v,p,a,alpha,mu
12969,0.019685,15.905512,0.0,-0.002201,0.006965,-0.086716,0.015988,0.0,7.220286
12970,0.629921,15.905512,0.0,-0.002405,0.007617,-0.092706,0.008958,0.0,7.023582
12971,1.240157,15.905512,0.0,-0.002768,0.008169,-0.098786,0.003086,0.0,6.818976
12972,1.850394,15.905512,0.0,-0.003635,0.009205,-0.104278,-0.000958,0.0,6.588605
12973,2.460630,15.905512,0.0,-0.005100,0.010284,-0.109445,-0.001718,0.0,6.399035
